In [9]:
import os
import eyed3
import spotipy
from dotenv import load_dotenv

In [10]:
# Get all the .mp3 files of the music folder

folder = 'H:/Musica'
files = [f for f in os.listdir(folder) if f.endswith('.mp3')]

print("MP3 files in the music folder: ", len(files))
print(files[:5])

MP3 files in the music folder:  882
['(I Just) Died in Your Arms Ton.mp3', "- 'Bohemian Rhapsody'.mp3", "- 'Don't Stop Me Now'.mp3", '- Aqualung.mp3', '- Beds Are Burning.mp3']


In [11]:
%%capture
# Get the metadata of every song file and save it in tuple title, artist, album
# Cell without output

data = []
failed = []
for f in files:
    try:
        audiofile = eyed3.load(folder + '/' + f)
        title = audiofile.tag.title
        artist = audiofile.tag.artist
        album = audiofile.tag.album
        data.append((title, artist, album))
    except:
        failed.append(f)

In [12]:
print("Number of files with metadata: ", len(data))
print("Number of files without metadata: ", len(failed), failed)

Number of files with metadata:  876
Number of files without metadata:  6 ['All That She Wants.mp3', 'M-O-O-N - Delay.mp3', 'Promise Me.mp3', 'Queen - I Want It All - Remastered 2011.mp3', 'Queen - I Want To Break Free - Remastered 2011 (1).mp3', 'Supertramp - Give A Little Bit.mp3']


In [13]:
# Spotify API
load_dotenv()
spotify_username = os.getenv('spotify_username')
spotify_client_id = os.getenv('SPOTIPY_CLIENT_ID')
spotify_client_secret = os.getenv('SPOTIPY_CLIENT_SECRET')
spotify_redirect_uri = os.getenv('SPOTIPY_REDIRECT_URI')
print(spotify_username, spotify_client_id, spotify_client_secret, spotify_redirect_uri)

aguimorefran 3722163c4c494db9bc4e4d68a48754b6 b33c57ccc6e741adb7732fb2bf7cf588 http://127.0.0.1:8080/


In [14]:
# Create the playlist
playlist_name = "ipodLocal"
scope = 'playlist-modify-public'
token = spotipy.util.prompt_for_user_token(spotify_username, scope, client_id=spotify_client_id, client_secret=spotify_client_secret, redirect_uri=spotify_redirect_uri)
sp = spotipy.Spotify(auth=token)

# Create if not exists
playlist_id = None
playlists = sp.user_playlists(spotify_username)
for playlist in playlists['items']:
    if playlist['name'] == playlist_name:
        playlist_id = playlist['id']
        print("Playlist already exists")
        break

if playlist_id is None:
    playlist = sp.user_playlist_create(spotify_username, playlist_name)
    playlist_id = playlist['id']
    print("Playlist created")

print("Playlist id: ", playlist_id)


Playlist already exists
Playlist id:  6CqnG0BN0WCdKnVtJyqj4r


In [17]:
# Search the songs in Spotify and add them to the playlist

# Tracks already added to the playlist
playlist_tracks = sp.user_playlist_tracks(spotify_username, playlist_id)
playlist_tracks_ids = [track['track']['id'] for track in playlist_tracks['items']]

failed_search = []
for idx, song in enumerate(data):
    title, artist, album = song
    if title is None:
        title = ''
    if artist is None:
        artist = ''
    query = 'artist:' + artist + ' track:' + title
    results = sp.search(q=query, limit=1)
    if len(results['tracks']['items']) > 0:
        track = results['tracks']['items'][0]
        track_id = track['id']
        # Check if the song is already in the playlist
        if track_id not in playlist_tracks_ids:
            sp.user_playlist_add_tracks(spotify_username, playlist_id, [track_id])

    else:
        # Clean name. Remove symbols and numbers
        title = ''.join(e for e in title if e.isalnum())
        query = 'artist:' + artist + ' track:' + title
        results = sp.search(q=query, limit=1)
        if len(results['tracks']['items']) > 0:
            track = results['tracks']['items'][0]
            track_id = track['id']
            sp.user_playlist_add_tracks(spotify_username, playlist_id, [track_id])
        else:
            failed_search.append(song)

    # Print the progress
    if idx % 10 == 0:
        print("Progress: ", idx, "/", len(data))

Progress:  0 / 876
Progress:  10 / 876
Progress:  20 / 876
Progress:  30 / 876
Progress:  40 / 876
Progress:  50 / 876
Progress:  60 / 876
Progress:  70 / 876
Progress:  80 / 876
Progress:  90 / 876
Progress:  100 / 876
Progress:  110 / 876
Progress:  120 / 876
Progress:  130 / 876
Progress:  140 / 876
Progress:  150 / 876
Progress:  160 / 876
Progress:  170 / 876
Progress:  180 / 876
Progress:  190 / 876
Progress:  200 / 876
Progress:  210 / 876
Progress:  220 / 876
Progress:  230 / 876
Progress:  240 / 876
Progress:  250 / 876
Progress:  260 / 876
Progress:  270 / 876
Progress:  280 / 876
Progress:  290 / 876
Progress:  300 / 876
Progress:  310 / 876
Progress:  320 / 876
Progress:  330 / 876
Progress:  340 / 876
Progress:  350 / 876
Progress:  360 / 876
Progress:  370 / 876
Progress:  380 / 876
Progress:  390 / 876
Progress:  400 / 876
Progress:  410 / 876
Progress:  420 / 876
Progress:  430 / 876
Progress:  440 / 876
Progress:  450 / 876
Progress:  460 / 876
Progress:  470 / 876
Pro

In [27]:
# Print the songs that couldn't be found
print("Songs not found: ", len(failed_search))

# Get tracks again
playlist_tracks = sp.user_playlist_tracks(spotify_username, playlist_id)
playlist_tracks_ids = [track['track']['id'] for track in playlist_tracks['items']]

# Remove parenthesis from the title and everything inside
for idx, song in enumerate(failed_search):
    title, artist, album = song
    if title is None:
        title = ''
    if artist is None:
        artist = ''
    if '(' in title:
        title = title[:title.index('(')]
    query = artist.strip() + ' ' + title.strip()
    results = sp.search(q=query, limit=1)
    if len(results['tracks']['items']) > 0:
        track = results['tracks']['items'][0]
        track_id = track['id']
        if track_id not in playlist_tracks_ids:
            sp.user_playlist_add_tracks(spotify_username, playlist_id, [track_id])
    
    # Print the progress
    if idx % 10 == 0:
        print("Progress: ", idx, "/", len(failed_search))

Songs not found:  104
Progress:  0 / 104
Progress:  10 / 104
Progress:  20 / 104
Progress:  30 / 104
Progress:  40 / 104
Progress:  50 / 104
Progress:  60 / 104
Progress:  70 / 104
Progress:  80 / 104
Progress:  90 / 104
Progress:  100 / 104


In [26]:
# Print the songs that couldn't be found
print("Songs not found: ", len(failed_search))

Songs not found:  104


In [29]:
# Remove repeated songs
playlist_tracks = sp.user_playlist_tracks(spotify_username, playlist_id)
playlist_tracks_ids = [track['track']['id'] for track in playlist_tracks['items']]
playlist_tracks_ids = list(set(playlist_tracks_ids))

# Remove repeated songs
sp.user_playlist_replace_tracks(spotify_username, playlist_id, playlist_tracks_ids)


{'snapshot_id': 'ODc3LDkwYzBiYWZmYzU0MjFmYTcyZWRiMzAyYWIxOWZjYjI0MDUwMmM4OTU='}